<a href="https://colab.research.google.com/github/endophenotype/Spark/blob/main/Spark_%D0%B7%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_%D0%BF%D0%B0%D0%BA%D0%B5%D1%82%D0%B0_%D1%81%D0%BE%D0%B1%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8_%D1%8D%D0%BA%D1%81%D0%BF%D0%BB%D1%83%D0%B0%D1%82%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.pandas as ps
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

**Реализовать код загрузки пакета на собственных данных и проэксплуатировать его, сделать выводы **

Фреймворк Spark может работать с данными, которые загружаются из различных файлов, рассмотрим наиболее популярные форматы:

.txt

.csv

.json

**.txt**

In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[*]'
conf = SparkConf().setAppName(appName).setMaster(master)
sc = spark.sparkContext
R = sc.textFile("/content/TG.txt");
print(R.collect())

['t1,g1', 't1,g2', 't2,g3', 't2,g4', 't3,g7', 't3,g8', 't3,g9']


**.csv**

In [ ]:
df = spark.read.csv('TG.csv', header=True, inferSchema=True)
df.show()

+------------------------+
|number;name;fmane;sname |
+------------------------+
|                 1;T;G;Y|
|                 2;T;G;Y|
|                 3;G;G;Y|
|                 4;T;G;Y|
|                 5;G;G;Y|
+------------------------+



**.json**

In [ ]:
# spark is an existing SparkSession
df = spark.read.json("/content/TG1.json")

# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()

# Select only the "name" column
df.show()

root
 |-- Array: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Boolean: boolean (nullable = true)
 |-- FName: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Null: string (nullable = true)
 |-- Number: long (nullable = true)
 |-- Object: struct (nullable = true)
 |    |-- a: string (nullable = true)
 |    |-- c: string (nullable = true)

+---------+-------+-------+----+----+------+------+
|    Array|Boolean|  FName|Name|Null|Number|Object|
+---------+-------+-------+----+----+------+------+
|[1, 2, 3]|   true|Golubev| Tim|null|   123|{b, d}|
+---------+-------+-------+----+----+------+------+



При помощи Spark можно работать с различными типами данных, выбор зависит от потребностей разработчика

**2. Реализовать код выбора и доступа к данным (DataFrame.select()) на собственных данных и проэксплуатировать его, сделать выводы **

In [ ]:
# spark is an existing SparkSession
df = spark.read.json("/content/TG1.json")

# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()

# Select only the "name" column
df.select("Name").show()
df.select("FName").show()

root
 |-- Array: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Boolean: boolean (nullable = true)
 |-- FName: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Null: string (nullable = true)
 |-- Number: long (nullable = true)
 |-- Object: struct (nullable = true)
 |    |-- a: string (nullable = true)
 |    |-- c: string (nullable = true)

+----+
|Name|
+----+
| Tim|
+----+

+-------+
|  FName|
+-------+
|Golubev|
+-------+



С помощью кода выбора и доступа к данным (DataFrame.select()) можно выбрать и работать с фрагментом данных

**Реализовать код Spark ввод-вывод на собственных данных и проэксплуатировать его, сделать выводы **

In [ ]:
data = spark.read.csv('TG.csv',inferSchema=True, header=True)
data.show()

+------------------------+
|number;name;fmane;sname |
+------------------------+
|                 1;T;G;Y|
|                 2;T;G;Y|
|                 3;G;G;Y|
|                 4;T;G;Y|
|                 5;G;G;Y|
+------------------------+



Запишем данные в новый файл

In [ ]:
data.write.csv("TimG.csv")

Проверим

In [ ]:
dfcheck = spark.read.csv('TimG.csv',inferSchema=True, header=True)
dfcheck.show()

+-------+
|1;T;G;Y|
+-------+
|2;T;G;Y|
|3;G;G;Y|
|4;T;G;Y|
|5;G;G;Y|
+-------+



Вывод: При помощи данного метода можно вводить данные в spark, проводить над ними манипуляции, а после выгрузить при помощи записи и дальнейшего вывода файла